<a href="https://colab.research.google.com/github/ankitagg2008/Misc/blob/main/Test_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip -q install diffusers transformers accelerate torch xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00


In [4]:
import torch
from PIL import Image
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler
from diffusers.utils import load_image

controlnet = ControlNetModel.from_pretrained("DionTimmer/controlnet_qrcode-control_v1p_sd15",
                                             torch_dtype=torch.float16)

pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.enable_xformers_memory_efficient_attention()
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def resize_for_condition_image(input_image: Image, resolution: int):
    input_image = input_image.convert("RGB")
    W, H = input_image.size
    k = float(resolution) / min(H, W)
    H *= k
    W *= k
    H = int(round(H / 64.0)) * 64
    W = int(round(W / 64.0)) * 64
    img = input_image.resize((W, H), resample=Image.LANCZOS)
    return img


# play with guidance_scale, controlnet_conditioning_scale and strength to make a valid QR Code Image

# qr code image
source_image = load_image("https://github.com/ankitagg2008/Misc/blob/efefc9b0e833a5da787b014e2e80bd082ffa69a0/test.png")
# initial image, anything
init_image = load_image("https://github.com/ankitagg2008/Misc/blob/c46968ffe2a2d8f4814d9c7bc3edc8929a03203a/test_1.png")
condition_image = resize_for_condition_image(source_image, 768)
init_image = resize_for_condition_image(init_image, 768)
generator = torch.manual_seed(123121231)
image = pipe(prompt="a bilboard in NYC with a qrcode",
             negative_prompt="ugly, disfigured, low quality, blurry, nsfw",
             image=init_image,
             control_image=condition_image,
             width=768,
             height=768,
             guidance_scale=20,
             controlnet_conditioning_scale=1.5,
             generator=generator,
             strength=0.9,
             num_inference_steps=150,
            )

image.images[0]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 35>:35                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/utils/testing_utils.py:275 in load_image       │
│                                                                                                  │
│   272 │   """                                                                                    │
│   273 │   if isinstance(image, str):                                                             │
│   274 │   │   if image.startswith("http://") or image.startswith("https://"):                    │
│ ❱ 275 │   │   │   image = PIL.Image.open(requests.get(image, stream=True).raw)                   │
│   276 │   │   elif os.path.isfile(image):                                                        │
│   277 │   │   │   image = PIL.Image.open(image)                                                  │
│   278 │   │   else:                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/PIL/Image.py:3030 in open                                │
│                                                                                                  │
│   3027 │   │   fp.close()                                                                        │
│   3028 │   for message in accept_warnings:                                                       │
│   3029 │   │   warnings.warn(message)                                                            │
│ ❱ 3030 │   raise UnidentifiedImageError(                                                         │
│   3031 │   │   "cannot identify image file %r" % (filename if filename else fp)                  │
│   3032 │   )                                                                                     │
│   3033                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f26e57f5350>